## Investigate Different PyBaMM Solvers

In this notebook, we discuss the process of changing PyBaMM solvers and the corresponding performance trade-offs with each. For further reading on different solvers, see the PyBaMM solver documentation:

[[1]: PyBaMM Solvers](https://docs.pybamm.org/en/stable/source/api/solvers/index.html#)

### Setting up the Environment

Before we begin, we need to ensure that we have all the necessary tools. We will install PyBOP from its development branch and upgrade some dependencies:

In [ ]:
%pip install --upgrade pip ipywidgets -q
%pip install pybop -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Importing Libraries

With the environment set up, we can now import PyBOP alongside other libraries we will need:

In [ ]:
import time

import numpy as np
import pybamm

import pybop

Let's fix the random seed in order to generate consistent output during development, although this does not need to be done in practice.

In [ ]:
np.random.seed(8)

### Generate Synthetic Data

To demonstrate parameter estimation, we first need some data. We will generate synthetic data using the PyBOP forward model, which requires defining a parameter set and the model itself.

#### Defining Parameters and Model

We start by creating an example parameter set and then instantiate the single-particle model (SPM):

In [ ]:
parameter_set = pybop.ParameterSet.pybamm("Chen2020")
model = pybop.lithium_ion.SPMe(parameter_set=parameter_set)

In [ ]:
t_eval = np.arange(0, 900, 2)
values = model.predict(t_eval=t_eval)

### Adding Noise to Voltage Data

To make the parameter estimation more realistic, we add Gaussian noise to the data.

In [ ]:
sigma = 0.001
corrupt_values = values["Voltage [V]"].data + np.random.normal(0, sigma, len(t_eval))

## Identify the Parameters
We will now set up the parameter estimation process by defining the datasets for optimisation and selecting the model parameters we wish to estimate.

### Creating Optimisation Dataset

The dataset for optimisation is composed of time, current, and the noisy voltage data:

In [ ]:
dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": values["Current [A]"].data,
        "Voltage [V]": corrupt_values,
    }
)

### Defining Parameters to Estimate

We select the parameters for estimation and set up their prior distributions and bounds:

In [ ]:
parameters = [
    pybop.Parameter(
        "Negative electrode active material volume fraction",
        prior=pybop.Gaussian(0.6, 0.02),
        bounds=[0.5, 0.8],
    ),
    pybop.Parameter(
        "Positive electrode active material volume fraction",
        prior=pybop.Gaussian(0.48, 0.02),
        bounds=[0.4, 0.7],
    ),
]

### Setting up the Optimisation Problem

With the datasets and parameters defined, we can set up the optimisation problem, its cost function, and the optimiser.

In [ ]:
solvers = dict(
    IDAKLU=pybamm.IDAKLUSolver(atol=1e-6, rtol=1e-6),
    Casadi_safe=pybamm.CasadiSolver(atol=1e-6, rtol=1e-6, mode="safe"),
    Casadi_fast=pybamm.CasadiSolver(atol=1e-6, rtol=1e-6, mode="fast"),
    Casadi_fast_with_events=pybamm.CasadiSolver(
        atol=1e-6, rtol=1e-6, mode="fast with events"
    ),
)

Let's setup a toy-problem to showcase the diferences. We run the forward model `n` times, for a variety of input values,

In [ ]:
n = 25  # Number of solves
inputs = [[x, y] for x, y in zip(np.linspace(0.45, 0.6, n), np.linspace(0.45, 0.6, n))]

Next, let's call the non-gradient based cost evaluate for each of the solvers and time the correseponding outcomes,

In [ ]:
parameter_set = pybop.ParameterSet.pybamm("Chen2020")
for solver in solvers.values():
    model = pybop.lithium_ion.SPMe(parameter_set=parameter_set, solver=solver)
    problem = pybop.FittingProblem(model, parameters, dataset)
    cost = pybop.SumSquaredError(problem)

    # Timing
    t1 = time.time()
    for i in range(n):
        cost.evaluate(inputs[i])
    print(f"Cost Evaluate for {solver.name}: {time.time()-t1}")

Cost Evaluate for IDA KLU solver: 3.354677200317383
Cost Evaluate for CasADi solver with 'safe' mode: 2.0228309631347656
Cost Evaluate for CasADi solver with 'fast' mode: 1.9543209075927734
Cost Evaluate for CasADi solver with 'fast with events' mode: 2.0191171169281006


Great, so the Casadi performance appears to be XX times better! That is, atleast for the non-gradient solutions. Next, let's repeat the same toy problem, but for the gradient-based cost evaluation,

In [ ]:
parameter_set = pybop.ParameterSet.pybamm("Chen2020")
for solver in solvers.values():
    model = pybop.lithium_ion.SPMe(parameter_set=parameter_set, solver=solver)
    problem = pybop.FittingProblem(model, parameters, dataset)
    cost = pybop.SumSquaredError(problem)

    # Timing
    t1 = time.time()
    for i in range(n):
        cost.evaluateS1(inputs[i])
    print(f"Cost Evaluate for {solver.name}: {time.time()-t1}")

Cost Evaluate for IDA KLU solver: 1.4089879989624023
Cost Evaluate for CasADi solver with 'safe' mode: 3.617908000946045
Cost Evaluate for CasADi solver with 'fast' mode: 3.5603108406066895
Cost Evaluate for CasADi solver with 'fast with events' mode: 3.537614107131958


Now this is interesting, so we have performance variation between the Casadi solvers and the IDAKLU. Given this result, it appears to the Casadi solver is most benefical for non-gradient optimisers, while IDAKLU has a large improve for gradient-based optimisers.

- Gradient compared to non-gradient methods
- No GPU methods in this notebook
- Import data or use model.predict?